In [1]:
import logging
import re
from urllib.parse import urljoin,urlencode
import multiprocessing
import time
import json
from os import makedirs
from os.path import exists
import pandas as pd
from lxml import etree

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ChromeOptions

logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s: %(message)s')

# 基础网址和爬取页码数量设定
BASE_URL='https://www.9fzt.com/marketCenter/aStockMarket.html?tab=0'
TOT_PAGEs=4

In [2]:
# %%
def get_browser():
    option=ChromeOptions()
    option.add_experimental_option('excludeSwitches',['enable-automation'])
    option.add_experimental_option('useAutomationExtension',False)
    # 设置不显式地显示浏览器
    option.add_argument('--headless')
    browser=webdriver.Chrome(options=option)
    browser.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument",
                            {'source': 'Object.defineProperty(navigator,"webdriver",{get:()=>undefind})'})
    browser.implicitly_wait(10)
    return browser


In [3]:
def parse_page(page_html,protol_type='https:'):
    pattern=re.compile('序号(.*)',re.S)
    stock_urls=re.findall(pattern,page_html)
    pattern=re.compile('<a href="(.*?)".*? class="bluelink ff_din-medium fw-500" target="_blank" rel="noopener">',re.S)
    stock_urls=re.findall(pattern,str(stock_urls))
    stock_urls=[urljoin(protol_type,url) for url in stock_urls]
    return stock_urls


In [4]:
def scrape_stock_list(browser,page_num):
    browser.get(BASE_URL)
    urls=[]
    WebDriverWait(browser,20,0.5).until(lambda browser:len(
        browser.find_element(By.XPATH,'//*[@id="__next"]/div/div[3]/div[2]/ul[20]/li[13]/span').text)>0)
    for page in range(page_num):
        urls+=(parse_page(browser.page_source))
        time.sleep(0.1)
        ac=ActionChains(browser)
        # 鼠标移动到下一页按钮上
        ac.move_to_element(browser.find_element_by_name('whj_nextPage')).perform()
        # 点击确定跳转至下一页
        ac.click(browser.find_element_by_name('whj_nextPage')).perform()
        time.sleep(0.1)
        WebDriverWait(browser,10,0.5).until(lambda browser: len(
            browser.find_element(By.XPATH,'//*[@id="__next"]/div/div[3]/div[2]/ul[20]/li[13]/span').text)>0)
    print(urls)
    return urls


In [5]:
def scrape_stockprice_page(browser,url):
    logging.info('scraping %s...',url)
    browser.get(url)
    WebDriverWait(browser,60,2).until(lambda browser: browser.find_element_by_id('stockprice').text != '--')
    return browser.page_source


In [6]:
def scrape_company_list(browser,urls):
    companys=[]
    for url in urls:
        logging.info('scraping %s...',url)
        browser.get(url)
        logging.info('scraping 股东研究...')
        browser.find_element_by_link_text('股东研究').click()
        browser.switch_to.window(browser.window_handles[1])
        WebDriverWait(browser,10,2).until(lambda browser: len(
           browser.find_element(By.XPATH,'//*[@id="sdgd"]/div[2]/div/div/section/div[2]/div/div/div/div/div/table/tbody/tr[1]/td[6]/span').text)>0)
        shareholder=browser.page_source
        browser.close()
        browser.switch_to.window(browser.window_handles[0])
        #shareholder = None

        companys.append([shareholder])
    return companys


In [7]:
# 使用xpath匹配
shareholer_name_xpath='//*[@id="SHNameTopTenSH0"]/text()'
shareholer_nature_xpath='//*[@id="SHNatureTopTenSH0"]/text()'
share_type_xpath='//*[@id="ShareTypeTopTenSH0"]/text()'
share_amount_xpath='//*[@id="sdgd"]/div[2]/div/div/section/div[2]/div/div/div/div/div/table/tbody/tr[1]/td[5]/span/text()'
share_ratio_xpath='//*[@id="sdgd"]/div[2]/div/div/section/div[2]/div/div/div/div/div/table/tbody/tr[1]/td[6]/span/text()'

def parse_company_data(html):
    html=etree.HTML(html)
    shareholer_name=html.xpath(shareholer_name_xpath)[0]
    shareholer_nature=html.xpath(shareholer_nature_xpath)[0]
    share_type=html.xpath(share_type_xpath)[0]
    share_amount=html.xpath(share_amount_xpath)[0]
    share_ratio=html.xpath(share_ratio_xpath)[0]

    return [shareholer_name,shareholer_nature,share_type,share_amount,share_ratio]


In [8]:
def get_table_company_data(data):
    table=pd.DataFrame(columns=['shareholer_name','shareholer_nature','share_type','share_amount','share_ratio'])
    for item in data:
        table.loc[len(table)]=parse_company_data(item)
    return table


In [9]:
browser=get_browser()
urls=scrape_stock_list(browser,TOT_PAGEs)
#urls=['https://stock.9fzt.com/index/bj_870204.html']

companys=scrape_company_list(browser,urls)
data=[company[0] for company in companys]

company_data_table=get_table_company_data(data)
#print(company_data_table)

company_data_table.to_csv('share_data_table.csv',index=False,encoding='gbk')


2023-07-04 19:20:04,097 - INFO: scraping https://stock.9fzt.com/index/sz_301488.html...


['https://stock.9fzt.com/index/sz_301488.html', 'https://stock.9fzt.com/index/bj_833533.html', 'https://stock.9fzt.com/index/sz_301007.html', 'https://stock.9fzt.com/index/sz_301255.html', 'https://stock.9fzt.com/index/sz_300489.html', 'https://stock.9fzt.com/index/sz_301221.html', 'https://stock.9fzt.com/index/sz_300552.html', 'https://stock.9fzt.com/index/sh_688280.html', 'https://stock.9fzt.com/index/sz_300503.html', 'https://stock.9fzt.com/index/sz_300549.html', 'https://stock.9fzt.com/index/sh_688331.html', 'https://stock.9fzt.com/index/bj_831278.html', 'https://stock.9fzt.com/index/sh_688147.html', 'https://stock.9fzt.com/index/sh_688123.html', 'https://stock.9fzt.com/index/bj_836221.html', 'https://stock.9fzt.com/index/sh_688326.html', 'https://stock.9fzt.com/index/sz_301099.html', 'https://stock.9fzt.com/index/sh_688071.html', 'https://stock.9fzt.com/index/sz_300496.html', 'https://stock.9fzt.com/index/sz_301183.html', 'https://stock.9fzt.com/index/sz_301488.html', 'https://sto

2023-07-04 19:20:05,495 - INFO: scraping 股东研究...
2023-07-04 19:20:09,100 - INFO: scraping https://stock.9fzt.com/index/bj_833533.html...
2023-07-04 19:20:09,869 - INFO: scraping 股东研究...
2023-07-04 19:20:12,933 - INFO: scraping https://stock.9fzt.com/index/sz_301007.html...
2023-07-04 19:20:13,686 - INFO: scraping 股东研究...
2023-07-04 19:20:16,314 - INFO: scraping https://stock.9fzt.com/index/sz_301255.html...
2023-07-04 19:20:17,492 - INFO: scraping 股东研究...
2023-07-04 19:20:19,802 - INFO: scraping https://stock.9fzt.com/index/sz_300489.html...
2023-07-04 19:20:20,700 - INFO: scraping 股东研究...
2023-07-04 19:20:23,378 - INFO: scraping https://stock.9fzt.com/index/sz_301221.html...
2023-07-04 19:20:24,210 - INFO: scraping 股东研究...
2023-07-04 19:20:26,205 - INFO: scraping https://stock.9fzt.com/index/sz_300552.html...
2023-07-04 19:20:26,893 - INFO: scraping 股东研究...
2023-07-04 19:20:29,508 - INFO: scraping https://stock.9fzt.com/index/sh_688280.html...
2023-07-04 19:20:30,798 - INFO: scraping 

2023-07-04 19:24:51,671 - INFO: scraping 股东研究...
2023-07-04 19:24:58,959 - INFO: scraping https://stock.9fzt.com/index/sh_603297.html...
2023-07-04 19:25:00,933 - INFO: scraping 股东研究...
2023-07-04 19:25:06,918 - INFO: scraping https://stock.9fzt.com/index/sh_603786.html...
2023-07-04 19:25:08,552 - INFO: scraping 股东研究...
2023-07-04 19:25:16,541 - INFO: scraping https://stock.9fzt.com/index/sz_002997.html...
2023-07-04 19:25:18,690 - INFO: scraping 股东研究...
2023-07-04 19:25:25,101 - INFO: scraping https://stock.9fzt.com/index/sh_603266.html...
2023-07-04 19:25:27,914 - INFO: scraping 股东研究...
2023-07-04 19:25:34,485 - INFO: scraping https://stock.9fzt.com/index/sh_603355.html...
2023-07-04 19:25:36,145 - INFO: scraping 股东研究...
2023-07-04 19:25:43,051 - INFO: scraping https://stock.9fzt.com/index/sh_605005.html...
2023-07-04 19:25:45,456 - INFO: scraping 股东研究...
2023-07-04 19:25:50,541 - INFO: scraping https://stock.9fzt.com/index/sz_002232.html...
2023-07-04 19:25:52,911 - INFO: scraping 